In [ ]:
import pandas as pd
import re
import nltk as nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import time
import math
import string
import gensim
#import pyLDAvis
import os
import requests
import pprint
import seaborn as sns
import matplotlib.pyplot as plt
#import pyLDAvis.gensim
#import pyLDAvis.gensim_models as gensimvis
import warnings
warnings.simplefilter('ignore')


from IPython.display import clear_output
from joblib import dump, load
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore

#pyLDAvis.enable_notebook()
#nlp = spacy.load("en_core_web_sm")

In [ ]:
!pip install pandarallel

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.8 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16673 sha256=11ac1ed104f2f43458ccbd162485475607a0d71347b41bb67bc82caa7aeb8ffe
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel


In [ ]:
import pandarallel
import multiprocessing
from pandarallel import pandarallel

num_processors = multiprocessing.cpu_count()

pandarallel.initialize(nb_workers=num_processors, use_memory_fs=False, progress_bar=True)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


### Sentiment Analysis (Training the Yelp Custom Sentiment Model)

In [ ]:
def get_gcs_data (bucket_name, folder_name, file_name, path_local):
    url = 'https://storage.googleapis.com/' + bucket_name + '/' + folder_name + '/' + file_name
    r = requests.get(url)
    open(path_local + '/' + file_name , 'wb').write(r.content)

In [ ]:
path_yelp_data = '/home/jupyter/yelp'
path_yelp_model = '/home/jupyter/data/yelp/yelp_model/'

os.makedirs(path_yelp_data, exist_ok=True)
os.makedirs(path_yelp_model, exist_ok=True)

In [ ]:
bucket_name = 'msca-bdp-data-open'
folder_name = 'yelp/yelp_model'
file_name = ['model.joblib',
             'nb.joblib', 'logreg.joblib', 'svm.joblib',
             # 'nb_small.joblib', 'logreg_small.joblib', 'svm_small.joblib'
            ]
path_local = path_yelp_model

os.makedirs(path_local, exist_ok=True)

for file in file_name:
    get_gcs_data (bucket_name = bucket_name,
                 folder_name = folder_name,
                 file_name = file,
                 path_local = path_local)
    print('Downloaded: ' + file)

Downloaded: model.joblib
Downloaded: nb.joblib
Downloaded: logreg.joblib
Downloaded: svm.joblib


In [ ]:
!ls -l /home/jupyter/data/yelp/yelp_model/

total 1848488
-rw-r--r-- 1 root root 522995808 Aug 17 08:27 logreg.joblib
-rw-r--r-- 1 root root   7661465 Aug 17 08:27 model.joblib
-rw-r--r-- 1 root root 839186606 Aug 17 08:27 nb.joblib
-rw-r--r-- 1 root root 522996092 Aug 17 08:27 svm.joblib


In [ ]:
# Logistic Regression
%time clf_large = load(os.path.join(path_yelp_model, 'logreg.joblib'))

CPU times: user 54.8 s, sys: 3.13 s, total: 57.9 s
Wall time: 58 s


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/filter_data.parquet'

In [ ]:
%%time

df = pd.read_parquet(file_path)
df.shape

CPU times: user 240 ms, sys: 211 ms, total: 450 ms
Wall time: 485 ms


(1650, 13)

In [ ]:
X_test = df['clean_text']

In [ ]:
%%time

y_pred = clf_large.predict(X_test)

CPU times: user 2.71 s, sys: 6.27 ms, total: 2.72 s
Wall time: 2.75 s


In [ ]:
df['sentiment'] = y_pred

In [ ]:
df[['clean_text', 'sentiment']].sample(10, random_state = 42)

,clean_text,sentiment
482,Standigm Signs MOU with Merck Korea for AI dru...,0
65,Global Artificial Intelligence in the Educatio...,1
926,Better Therapeutics gets US patent linked to m...,0
481,Microsoft Bing getting ChatGPTlike conversatio...,0
23,Continue AI aims to add ESG intelligence to su...,0
1089,"AXES DONATES 150,000 to AI RESEARCH AT UNLVSki...",0
561,Ferrum Health Raises 6 Million to Bring Artifi...,0
259,Google expands its conversational Duplex AI to...,1
1278,INTRODUCING NETRAMARK A PHARMA AI COMPANYSkip ...,0
124,Quantum Music to Showcase Groundbreaking AI Ba...,1


In [ ]:
# 0 is negative, 1 is positive
df['sentiment'].value_counts()

0    986
1    664
Name: sentiment, dtype: int64

In [ ]:
!pip install pyarrow

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

# Assuming 'df' is your DataFrame
file_path = '/content/filter_data_senti.parquet'
# Convert DataFrame to PyArrow Table
table = pa.Table.from_pandas(df)

# Write the PyArrow Table to a Parquet file
pq.write_table(table, file_path)

In [ ]:
from google.colab import files

files.download('/content/filter_data_senti.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>